# Deep Learning with Neural Networks and TensorFlow

- We're going to be working first with the MNIST dataset, which is a dataset that contains 60,000 training samples and 10,000 testing samples of hand-written and labeled digits, 0 through 9, so ten total "classes."
- The MNIST dataset has the images, which we'll be working with as purely black and white, thresholded, images, of size 28 x 28, or 784 pixels total. Our features will be the pixel values for each pixel, thresholded.

# Detail steps

__1. Feed forward:__
- First, we take our input data, and we need to send it to hidden layer 1. Thus, we weight the input data, and send it to layer 1, where it will undergo the activation function, so the neuron can decide whether or not to fire and output some data to either the output layer, or another hidden layer. We will have three hidden layers in this example, making this a Deep Neural Network.

__2. Compare output to intended output:__
- We will use a cost function (alternatively called a loss function), to determine how wrong we are. Example: cross entropy
    
__3. optimizer function:__
- We will use an optimizer function (Adam Optimizer, SGD, AdaGrad, ...) to minimize the cost.
- The way cost is minimized is by tinkering with the weights, with the goal of hopefully lowering the cost. How quickly we want to lower the cost is determined by the learning rate. The lower the value for learning rate, the slower we will learn, and the more likely we'll get better results.

__4. Backpropagation__


- The act of sending the data straight through our network means we're operating a __feed forward neural network__. The adjusting of weights backwards is our __back propagation__.
- The cycle is caled an __epoch__ (feed forward + backpropagate)


In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
n_nodes_hl1 = 500 # hidden layer 1
n_nodes_hl2 = 500 # hidden layer 2
n_nodes_hl3 = 500 # hidden layer 3
n_classes = 10
batch_size = 100

In [9]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

## Build a neural network

In [10]:
def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}

    # (input_data * weights) + biases
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

- (input_data * weights) + biases
- The bias is a value that is added to our sums, before being passed through the activation function.
- The purpose of the bias here is mainly to handle for scenarios where all neurons fired a 0 into the layer. Specifically if input_data is 0 then (input_data * weights) = 0, then no neurons would fire. A bias makes it possible that a neuron still fires out of that layer. A bias is as unique as the weights, and will need to be optimized too.

## Train a neural network

In [11]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # how many epochs (cycles of feed forward and back prop)
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [12]:
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 10, 'loss:', 1809674.8974075317)
('Epoch', 1, 'completed out of', 10, 'loss:', 409088.21184635162)
('Epoch', 2, 'completed out of', 10, 'loss:', 225860.59725761414)
('Epoch', 3, 'completed out of', 10, 'loss:', 133422.67149591446)
('Epoch', 4, 'completed out of', 10, 'loss:', 82367.870904112613)
('Epoch', 5, 'completed out of', 10, 'loss:', 51699.77699768007)
('Epoch', 6, 'completed out of', 10, 'loss:', 36577.346919721364)
('Epoch', 7, 'completed out of', 10, 'loss:', 24832.629106469743)
('Epoch', 8, 'completed out of', 10, 'loss:', 20779.564563903703)
('Epoch', 9, 'completed out of', 10, 'loss:', 18933.010037306522)
('Accuracy:', 0.95160002)


# References
- https://pythonprogramming.net/tensorflow-deep-neural-network-machine-learning-tutorial/
- https://pythonprogramming.net/tensorflow-neural-network-session-machine-learning-tutorial/?completed=/tensorflow-deep-neural-network-machine-learning-tutorial/